In [9]:
#!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

In [10]:
import math
import cv2 as cv

def get_distance(p1,p2):
    return math.sqrt(((p2[0]-p1[0])**2+(p2[1]-p1[1])**2)+(p2[2]-p1[2])**2)

def checkPiedraPapelTijera(image,landmarks,x,y):
        if isHigher(8,6,landmarks) and isHigher(12,10,landmarks) and isHigher(16,14,landmarks) and isHigher(20,18,landmarks):
            cv.rectangle(image, x, y, (0, 255, 0), 5)
            cv.putText(image, "PIEDRA", x,  cv.FONT_HERSHEY_SIMPLEX, 2, (0,255,0),2)
            return
        elif isHigher(6,8,landmarks) and isHigher(10,12,landmarks) and isHigher(14,16,landmarks) and isHigher(18,20,landmarks):
            cv.rectangle(image, x, y, (255, 0, 0), 5)
            cv.putText(image, "PAPEL", x,  cv.FONT_HERSHEY_SIMPLEX, 2, (255,0,0),2)
            return
        elif isHigher(7,8,landmarks) and isHigher(11,12,landmarks) and isHigher(16,14,landmarks) and isHigher(20,18,landmarks):
            cv.rectangle(image, x, y, (0, 0, 255), 5)
            cv.putText(image, "TIJERA", x,  cv.FONT_HERSHEY_SIMPLEX, 2, (0,0,255),2)
            return

def isHigher(a,b,landmarks):
    return get_distance(landmarks[0],landmarks[a]) < get_distance(landmarks[0],landmarks[b])

import mediapipe as mp
import cv2 as cv

video = cv.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    while(True):
        ret, frame = video.read()
        h, w, c = frame.shape
        if ret == True:
            x_max = 0
            y_max = 0
            x_min = w
            y_min = h
            frame = cv.flip(frame, 1)
            cv.imshow("Salida", frame)

            # Flip the image horizontally for a later selfie-view display
            image = cv.cvtColor(cv.flip(frame, 1), cv.COLOR_BGR2RGB)
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            # Process the image and find hand landmarks
            results = hands.process(image)

            # Draw the hand landmarks on the image
            image.flags.writeable = True
            image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    points=[]
                    for point in hand_landmarks.landmark:
                        x, y = int(point.x * w), int(point.y * h)
                        points.append((point.x,point.y,point.z))
                        if x > x_max:
                            x_max = x
                        if x < x_min:
                            x_min = x
                        if y > y_max:
                            y_max = y
                        if y < y_min:
                            y_min = y
                    cv.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 5)
                res = checkPiedraPapelTijera(image,points)
                if res == "PIEDRA":
                    #cv.putText(image, "Prueba de texto", (10, 300),  cv.FONT_HERSHEY_SIMPLEX, 10, (0,255,0),12)
                    None
            cv.imshow('MediaPipe Hands', image)
        if ret == False:
            video.set(cv.CAP_PROP_POS_FRAMES, 0)
        if cv.waitKey(10) & 0xFF == 27: break

video.release()
cv.destroyAllWindows()

In [11]:
#@markdown We implemented some functions to visualize the hand landmark detection results. <br/> Run the following cell to activate the functions.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.

  return annotated_image

In [14]:
import cv2 as cv
import mediapipe as mp
import time
import pathlib

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

path=str(pathlib.Path().resolve())+"/"

options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=path+"hand_landmarker.task"),
    running_mode=VisionRunningMode.VIDEO,
    num_hands=int(1e5),
    min_hand_detection_confidence=0.3,
    min_hand_presence_confidence=0.3,
    min_tracking_confidence=0.3)


with HandLandmarker.create_from_options(options) as landmarker:

  t = time.time()
  video = cv.VideoCapture(0)

  while(True):

    ret, frame = video.read()
    h, w, c = frame.shape

    if ret == True:

      mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

      hand_landmarker_result = landmarker.detect_for_video(mp_image,int(round((time.time())*1000)))

      # Usa draw_landmarks para dibujar las landmarks y las conexiones en la imagen
      annotated_image=draw_landmarks_on_image(mp_image.numpy_view(), hand_landmarker_result)
      for landmark in hand_landmarker_result.hand_landmarks:
        for hands in hand_landmarker_result.hand_landmarks:
            x_max = 0
            y_max = 0
            x_min = w
            y_min = h
            points=[]
            for point in hands:
                x, y = int(point.x * w), int(point.y * h)
                points.append((point.x,point.y,point.z))
                if x > x_max:
                    x_max = x
                if x < x_min:
                    x_min = x
                if y > y_max:
                    y_max = y
                if y < y_min:
                    y_min = y
        #cv.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 5)
            checkPiedraPapelTijera(annotated_image,points,(x_min,y_min),(x_max,y_max))
         

      # Muestra la imagen con las landmarks
      cv.imshow("Prueba",annotated_image)
    if ret == False:
        video.set(cv.CAP_PROP_POS_FRAMES, 0)
    if cv.waitKey(10) & 0xFF == 27: break

video.release()
cv.destroyAllWindows()